In [1]:
import json
import os
import pandas as pd
from datetime import datetime
import pytz
import itertools

In [2]:
GAME = "testlauf-neupfarrplatz"

In [3]:
df = pd.read_parquet(os.path.join("../data", GAME, "log-interpol/interpol.parquet"))

In [4]:
df.head()

team 50b848eb-150d-11ef-a477-15bc960d382e             \
dim                                   lat        lon   
0                               49.018418  12.096596   
1000                            49.018418  12.096595   
2000                            49.018418  12.096595   
3000                            49.018418  12.096595   
4000                            49.018418  12.096595   

team a4eebb09-1514-11ef-b019-73f1875d90c5             \
dim                                   lat        lon   
0                               49.018377  12.096603   
1000                            49.018377  12.096603   
2000                            49.018377  12.096603   
3000                            49.018377  12.096603   
4000                            49.018377  12.096603   

team b1d91f00-1514-11ef-90d9-870e7bed4f26             \
dim                                   lat        lon   
0                               49.018386  12.096590   
1000                            49.018388  12.096587   
2000                            49.018388  12.096585   
3000                            49.018388  12.096585   
4000                            49.018388  12.096584   

team 56971933-150d-11ef-9427-2bf640545472             \
dim                                   lat        lon   
0                               49.018445  12.096630   
1000                            49.018444  12.096632   
2000                            49.018442  12.096636   
3000                            49.018440  12.096640   
4000                            49.018438  12.096644   

team 284f0e6b-1514-11ef-a3d1-710a0d2c97dd             
dim                                   lat        lon  
0                               49.018446  12.096713  
1000                            49.018445  12.096713  
2000                            49.018445  12.096713  
3000                            49.018445  12.096713  
4000                            49.018445  12.096714

In [5]:
# get list of teams
teams = list(set(c[0] for c in df.columns))
teams

['a4eebb09-1514-11ef-b019-73f1875d90c5',
 '50b848eb-150d-11ef-a477-15bc960d382e',
 '56971933-150d-11ef-9427-2bf640545472',
 'b1d91f00-1514-11ef-90d9-870e7bed4f26',
 '284f0e6b-1514-11ef-a3d1-710a0d2c97dd']

In [6]:
# get catch data
data_caught = json.load(open('../data/testlauf-neupfarrplatz/log-export/team_caught.json'))

In [7]:
# dictionary of caught timestamps
caught_timestamps = {}
for entry in data_caught:
    entry['timestamp'] = entry['timestamp'][:-4]
    entry['timestamp'] = entry['timestamp']
    dt = datetime.strptime(entry['timestamp'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    caught_timestamps[entry['runaway_active_user']] = int(dt.timestamp() * 1000)
# initial hunting team
entry = data_caught[-1]
caught_timestamps

{'56971933-150d-11ef-9427-2bf640545472': 1716041758000,
 'b1d91f00-1514-11ef-90d9-870e7bed4f26': 1716041166000,
 'a4eebb09-1514-11ef-b019-73f1875d90c5': 1716039541000}

In [8]:
# get initial hunter
initial_hunter = entry['hunter_active_user']
caught_timestamps[initial_hunter] = 0

In [9]:
interesting_timestamps = []
interesting_start = 0
interesting = False
for t in df.index:
    hunters = [initial_hunter] + [team for team in teams if team in caught_timestamps.keys() and caught_timestamps[team] < t]
    chasers = [team for team in teams if team not in hunters]
    
    # calculate pairwise distances between hunters and chasers
    still_interesting = False
    for hunter, chaser in itertools.product(hunters, chasers):
        d_lat = df[hunter, 'lat'][t] - df[chaser, 'lat'][t]
        d_lon = df[hunter, 'lon'][t] - df[chaser, 'lon'][t]
        d = ((d_lat*1000)**2 + (d_lon*1000)**2)**0.5
        if d < 0.5:
            if not interesting:
                interesting_start = t
            still_interesting = True
            interesting = True
            break
    
    # end of interesting period
    if interesting and not still_interesting:
        interesting_timestamps.append((interesting_start, t))
        interesting = False
        
interesting_timestamps.append((interesting_start, t))

In [10]:
interesting_timestamps

[(0, 366000),
 (2166000, 2439000),
 (2567000, 2602000),
 (2627000, 2638000),
 (2948000, 3132000),
 (3576000, 3660000),
 (3713000, 3718000),
 (3719000, 3725000),
 (3786000, 3835000),
 (3874000, 3965000),
 (3972000, 4222000),
 (4338000, 4462000)]